In [ ]:
import bpy
import numpy as np

def import_point_cloud(object_name, points_coords, faces):
    # Importing points cloud
    new_mesh = bpy.data.meshes.new(object_name + "_mesh")
    new_obj = bpy.data.objects.new(object_name, new_mesh)
    
    # For mesh filling
    edges = []
    
    # coords = list(tuple, tuple, ...)
    new_mesh.from_pydata(points_coords, edges, faces)
    
    # Misc
    new_obj.show_name = False
    new_mesh.update()
    return new_obj

def form_points(matrix):
    points = []
    rows = matrix.shape[0]
    cols = matrix.shape[1]
    for row in range(rows):
        for col in range(cols):
            if (matrix[row][col] == 0):
                continue
            
            y_coord = row/rows
            x_coord = col/cols
            z_coord = 0
            points.append([x_coord, y_coord, z_coord])
    return points

def check_point(row, col, matrix):
    if (matrix[row][col] != 0 and matrix[row+1][col] != 0 and matrix[row][col+1] != 0 and matrix[row+1][col+1] != 0):
        return True
    return False

def calc_point_index(row, col, matrix, points):
    rows = matrix.shape[0]
    cols = matrix.shape[1]
    index = 0
    for p in points:
        if (p[0] == col/cols and p[1] == row/rows):
            break
        index += 1
        
    return index

def form_faces(matrix):
    faces = []
    rows = matrix.shape[0]
    cols = matrix.shape[1]
    for row in range(rows-1):
        for col in range(cols - 1):
            check_flag = check_point(row, col, matrix)
            if (check_flag == True):
                face = [
                    calc_point_index(row, col, matrix),
                    calc_point_index(row, col+1, matrix),
                    calc_point_index(row+1, col+1, matrix),
                    calc_point_index(row+1, col, matrix)
                ]
                faces.append(face)
    return faces
        
density_matrix = np.loadtxt('Analytic_Optim.txt')

points = form_points(density_matrix)
faces = form_faces(density_matrix)

# Object creation
obj_from_PC = import_point_cloud("Topology", points, faces)

# Linking
bpy.context.collection.objects.link(obj_from_PC)